<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#How-to-build-a-rigid-avatar" data-toc-modified-id="How-to-build-a-rigid-avatar-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>How to build a rigid avatar</a></span><ul class="toc-item"><li><span><a href="#2D-Case" data-toc-modified-id="2D-Case-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>2D Case</a></span></li><li><span><a href="#3D-Case" data-toc-modified-id="3D-Case-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>3D Case</a></span></li></ul></li></ul></div>

# How to build a rigid avatar

F. Dubois - 2016

All avatars are made of nodes, elements, contactors and need a material and a model.

## 2D Case

In [ ]:
#Initialization
from pylmgc90 import pre
# 2D
dim = 2


Defining a material suitable for rigid objects. 

In [ ]:
mat = pre.material(name='TDURx',materialType='RIGID',density=1000.)
#help(mat)

#possible options:
print(pre.config.lmgc90dicts.bulkBehavOptions.keys())
# see parameters for 'RIGID'
print('options for RIGID:')
print(pre.config.lmgc90dicts.bulkBehavOptions['RIGID'])

Defining a model suitable for rigid objects.

In [ ]:
mod = pre.model(name='rigid', physics='MECAx', element='Rxx2D', dimension=dim)

#possible options:
print(pre.config.lmgc90dicts.modelOptions.keys())
# see parameters for 'MECAx'
print('options for MECAx:')
print(pre.config.lmgc90dicts.modelOptions['MECAx'])

Defining an empty avatar

In [ ]:
disk = pre.avatar(dimension=2)

## some introspection
## what is disk ? 
#help(disk)
## what is nodes in disk ? 
#help(disk.nodes)
## what are the keys in nodes ? 
#print(disk.nodes.keys())
## what are the values in nodes ? 
#print(disk.nodes.values())

Creating a node and adding it to the avatar (number MUST be 1)

In [ ]:
import numpy as np

help(pre.node)
no = pre.node(coor=np.array([0.,0.1]),number=1)

## some introspection
## what is no ? 
#help(no)
## what is containing no ? 
#dir(no)
#print no.coor

# adding no to avatar
disk.addNode( no )

## some introspection
## does no.nodes change ?
#print(disk.nodes.keys())
#print(disk.nodes.values())
## key to access to value of first node stored
#id=disk.nodes.keys()[0]
#print('id= ',id)
## what is this value ?
#help(disk.nodes[id])
#dir(disk.nodes[id])
#print(disk.nodes[id].coor)
#print(disk.nodes[id].getCoor())
#print(disk.nodes[id].number)
#print(disk.nodes.values()[0].coor)

Creating an element and adding it to the avatar

In [ ]:
disk.addBulk( pre.rigid2d() )

## some introspection
#help(disk.bulks)
#print(disk.bulks.keys())
#id=disk.bulks.keys()[0]
#help(disk.bulks[id])
#dir(disk.bulks[id])
#print(disk.bulks[id].connectivity)

 We need groups to add model/material, contactors, drivendof, etc.

In [ ]:
## does my object contain groups ?
#dir(disk.groups)
#print(disk.groups.keys())

# 
disk.defineGroups()

## does my object contain groups ?
#print(disk.groups.keys())
## what is group 'all'?
#help(disk.groups['all'])
#dir(disk.groups['all'])
#help(disk.groups['all'].bulks)
#dir(disk.groups['all'])
#print(disk.groups['all'].bulks.keys())
#print(disk.groups['all'].bulks[0])
#print(disk.groups['all'].nodes.keys())
#print(disk.groups['all'].contactors.keys())

Adding model and material (default added to 'all').

In [ ]:
disk.defineModel(model=mod)
disk.defineMaterial(material=mat)


Adding contactor (default added to 'all')

In [ ]:
con = 'DISKx'
if con == 'DISKx':
    disk.addContactors(shape='DISKx', color='BLUEx', byrd=1.)
elif con == 'DISKb': 
    disk.addContactors(shape='DISKx', color='BLUEx', byrd=1., shift=[1., 0.])
elif con == 'xKSID':
    disk.addContactors(shape='xKSID', color='BLUEx', byrd=1.)
elif con == 'JONCx':
    disk.addContactors(shape='JONCx', color='WALLx', axe1=1., axe2=0.01)
elif con == 'POLYG':
    disk.addContactors(shape='POLYG', color='REDxx', nb_vertices=4, 
                       vertices=np.array([[-1.,-1.],[1.,-1.],[1.,1.],[-1.,1.]]))
elif con == 'PT2Dx':
    disk.addContactors(shape='PT2Dx', color='VERTx', shift=[1., 0.])

# possible contactors
print(pre.config.lmgc90dicts.contactorOptions.keys())
# options for 'DISKx'
print(pre.config.lmgc90dicts.contactorOptions['DISKx'])

Needs to compute some internal values (mass, inertia, frame, etc)

In [ ]:
disk.computeRigidProperties()

Some magic functions

In [ ]:
disk2=pre.rigidDisk(r=1., center=[2., 2.], model=mod, material=mat, color='BLUEx') 

#Draw
bodies = pre.avatars()
bodies.addAvatar(disk)
bodies.addAvatar(disk2)
pre.visuAvatars(bodies)

In [ ]:
body=pre.rigidDiscreteDisk(r=1., center=[0.,0.], model=mod, material=mat, color='BLUEx')

#Draw
bodies = pre.avatars()
bodies.addAvatar(body)
pre.visuAvatars(bodies)

In [ ]:
body1=pre.rigidJonc(axe1=1., axe2=0.5, center=[0.,0.], model=mod, material=mat, color='BLUEx')
body2=pre.roughWall(l=1., r=0.5, center=[0.,2.], model=mod, material=mat, theta=0., 
                   color='WALLx', nb_vertex=5)
body3=pre.roughWall(l=1., r=0.5, center=[5.,2.], model=mod, material=mat, theta=0., 
                    color='WALLx', nb_vertex=0)
body4=pre.fineWall(l=1., r=0.5, center=[0.,4.], model=mod, material=mat, theta=0., 
                   color='WALLx', nb_vertex=10)
body5=pre.fineWall(l=1., r=0.5, center=[5.,4.], model=mod, material=mat, theta=0., 
                   color='WALLx', nb_vertex=0)
body6=pre.smoothWall(l=1., h=0.5, center=[0.,6.], nb_polyg=4, model=mod, material=mat, 
                     theta=0., color='WALLx')
body7=pre.granuloRoughWall(l=1., rmin=0.01, rmax=0.1, center=[0.,8.], model=mod, material=mat, 
                           theta=0., color='WALLx', nb_vertex=6)
body8=pre.granuloRoughWall(l=1., rmin=0.01, rmax=0.1, center=[5.,8.], model=mod, material=mat, 
                           theta=0., color='WALLx', nb_vertex=0)
#Draw
bodies = pre.avatars()
bodies.addAvatar(body1)
bodies.addAvatar(body2)
bodies.addAvatar(body3)
bodies.addAvatar(body4)
bodies.addAvatar(body5)
bodies.addAvatar(body6)
bodies.addAvatar(body7)
bodies.addAvatar(body8)
pre.visuAvatars(bodies)


In [ ]:
body1=pre.rigidPolygon(center=[1.,1.],model=mod, material=mat, theta=0., color='BLUEx', 
                       generation_type='regular', nb_vertices=5, radius=1.)
body2=pre.rigidPolygon(center=[3.,3.],model=mod, material=mat, theta=0., color='BLUEx',
                       generation_type='full', 
                       vertices=np.array([[-1.,-1.],[1.,-1.],[1.,1.],[-1.,1.]]))
#Draw
bodies = pre.avatars()
bodies.addAvatar(body1)
bodies.addAvatar(body2)
pre.visuAvatars(bodies)


In [ ]:
body=pre.rigidOvoidPolygon(ra=1., rb=2., nb_vertices=10, center=[0.,0.], 
                           model=mod, material=mat, theta=0., color='BLUEx')
#Draw
bodies = pre.avatars()
bodies.addAvatar(body)
pre.visuAvatars(bodies)


Building a cluster

In [ ]:
boarf = pre.avatar(dimension=2)
no = pre.node(coor=np.array([0.,0.]),number=1)
boarf.addNode( no )
boarf.addBulk( pre.rigid2d() )
boarf.defineGroups()
boarf.defineModel(model=mod)
boarf.defineMaterial(material=mat)
boarf.addContactors(shape='POLYG', color='REDxx', nb_vertices=4, 
                    vertices=np.array([[-2.,-2.],[2.,-2.],[2.,-1.],[-2.,-1.]]))
boarf.addContactors(shape='POLYG', color='REDxx', nb_vertices=4, 
                    vertices=np.array([[-2.,-1.],[-1.,-1],[-1.,1.],[-2.,1.]]))
boarf.addContactors(shape='POLYG', color='REDxx', nb_vertices=4, 
                    vertices=np.array([[1.,-1.],[2.,-1.],[2.,1.],[1.,1.]]))
boarf.addContactors(shape='POLYG', color='REDxx', nb_vertices=4, 
                    vertices=np.array([[-2,1.],[2.,1.],[2.,2.],[-2.,2.]]))
boarf.addContactors(shape='DISKx', color='BLUEx', byrd=1.,shift=[3.,0.])
boarf.addContactors(shape='DISKx', color='BLUEx', byrd=1.,shift=[-3.,0.])
boarf.computeRigidProperties()
#Draw
bodies = pre.avatars()
bodies.addAvatar(boarf)
pre.visuAvatars(bodies)

In [ ]:
import math
bodies = pre.avatars()
tt=0.
for i in range(10):    
    boarf=pre.rigidDisk(r=1., center=[2*math.cos(tt), 2.*math.sin(tt)], 
                        model=mod, material=mat, color='BLUEx')
    bodies.addAvatar(boarf)
    tt+=math.pi/5
pre.visuAvatars(bodies)

Having more information on an object

In [ ]:
#pip3 install ppretty
from ppretty import ppretty
print(ppretty(boarf, indent='    ', width=40, seq_length=10, show_static=True, show_properties=True, show_address=False))

## 3D Case

Its almost the same than 2D !

In [ ]:
import numpy as np
from pylmgc90 import pre
dim=3
#
mod = pre.model(name='rigid', physics='MECAx', element='Rxx3D', dimension=dim)
mat = pre.material(name='TDURx',materialType='RIGID',density=1000.)
#
boarf = pre.avatar(dimension=3)
no = pre.node(coor=np.array([0.,0.,0.]),number=1)
boarf.addNode( no )
boarf.addBulk( pre.rigid3d() )
boarf.defineGroups()
boarf.defineModel(model=mod)
boarf.defineMaterial(material=mat)
#
con = 'POLYR'
if con == 'SPHER':
    boarf.addContactors(shape='SPHER', color='BLUEx', byrd=1.)
elif con == 'SPHERb':
    boarf.addContactors(shape='SPHER', color='BLUEx', byrd=1., shift=[1. , 0., 0.])
elif con == 'CYLND':
    boarf.addContactors(shape='CYLND', color='BLUEx', High=1., byrd=1.)
elif con == 'DNLYC':
    boarf.addContactors(shape='DNLYC', color='BLUEx', High=1., byrd=1.)
elif con == 'PLANx':
    boarf.addContactors(shape='PLANx', color='WALLx', axe1=0.5, axe2=1., axe3=0.01)
elif con == 'PLANb':
    boarf.addContactors(shape='PLANx', color='WALLx', axe1=0.5, axe2=1., axe3=0.01,
                        shift=[1., 0., 0.], 
                        frame=[[-1.,  0.,  0.],[ 0., -1.,  0.],[ 0.,  0.,  1.]] )
elif con == 'POLYR':
    # test des polyedres : un carre d'arete 1, centre en (0., 0.)
    #    * coordonnees des sommets (repere global)
    vertices = np.zeros([8, 3], 'd')
    #       - sommet 1
    vertices[0, 0]=-0.5
    vertices[0, 1]=-0.5
    vertices[0, 2]=-0.5
    #       - sommet 2
    vertices[1, 0]= 0.5
    vertices[1, 1]=-0.5
    vertices[1, 2]=-0.5
    #       - sommet 3
    vertices[2, 0]= 0.5
    vertices[2, 1]= 0.5
    vertices[2, 2]=-0.5
    #       - sommet 4
    vertices[3, 0]=-0.5
    vertices[3, 1]= 0.5
    vertices[3, 2]=-0.5
    #       - sommet 5
    vertices[4, 0]=-0.5
    vertices[4, 1]=-0.5
    vertices[4, 2]= 0.5
    #       - sommet 6
    vertices[5, 0]= 0.5
    vertices[5, 1]=-0.5
    vertices[5, 2]= 0.5
    #       - sommet 7
    vertices[6, 0]= 0.5
    vertices[6, 1]= 0.5
    vertices[6, 2]= 0.5
    #       - sommet 8
    vertices[7, 0]=-0.5
    vertices[7, 1]= 0.5
    vertices[7, 2]= 0.5
    #    * connectivite des faces
    faces = np.zeros([12, 3], 'i')
    faces[ 0, 0]=1; faces[ 0, 1]=2; faces[ 0, 2]=3
    faces[ 1, 0]=1; faces[ 1, 1]=3; faces[ 1, 2]=4
    faces[ 2, 0]=1; faces[ 2, 1]=2; faces[ 2, 2]=6
    faces[ 3, 0]=1; faces[ 3, 1]=6; faces[ 3, 2]=5
    faces[ 4, 0]=2; faces[ 4, 1]=3; faces[ 4, 2]=7
    faces[ 5, 0]=2; faces[ 5, 1]=7; faces[ 5, 2]=6
    faces[ 6, 0]=1; faces[ 6, 1]=4; faces[ 6, 2]=8
    faces[ 7, 0]=1; faces[ 7, 1]=8; faces[ 7, 2]=5
    faces[ 8, 0]=3; faces[ 8, 1]=4; faces[ 8, 2]=8
    faces[ 9, 0]=3; faces[ 9, 1]=8; faces[ 9, 2]=7
    faces[10, 0]=5; faces[10, 1]=7; faces[10, 2]=8
    faces[11, 0]=5; faces[11, 1]=6; faces[11, 2]=7 
    boarf.addContactors(shape='POLYR', color='REDxx', 
                        nb_vertices=8, vertices=vertices, 
                        nb_faces=12, connectivity=faces)
elif con == 'PT3Dx':    
    boarf.addContactors(shape='PT3Dx', color='VERTx', shift=[0., 1., 0.])

#Draw
bodies = pre.avatars()
bodies.addAvatar(boarf)
pre.visuAvatars(bodies)

Some magic functions

In [ ]:
body=pre.rigidSphere(r=1., center=[0.,0.,0.], model=mod, material=mat, color='BLUEx')
#Draw
bodies = pre.avatars()
bodies.addAvatar(body)
pre.visuAvatars(bodies)


In [ ]:

body=pre.rigidPlan(axe1=1., axe2=1., axe3=0.1, center=[0.,0.,0.], 
                   model=mod, material=mat, color='BLUEx')
#Draw
bodies = pre.avatars()
bodies.addAvatar(body)
pre.visuAvatars(bodies)


In [ ]:
body=pre.rigidCylinder(r=1., h=5., center=[0.,0.,0.], 
                       model=mod, material=mat, color='BLUEx', is_Hollow=False)
#Draw
bodies = pre.avatars()
bodies.addAvatar(body)
pre.visuAvatars(bodies)

In [ ]:
body = pre.rigidPolyhedron(model=mod, material=mat, center=np.zeros(3,'d'), generation_type='regular', 
                           color='BLUEx',nb_vertices=10, vertices=None, faces=None, radius=1.,
                           tol=0., number=None)
#Draw
bodies = pre.avatars()
bodies.addAvatar(body)
pre.visuAvatars(bodies)

In [ ]:
body=pre.roughWall3D(center=[0.,0.,0.], lx=2., ly=2., r=0.5, model=mod, material=mat, color='WALLx')
#Draw
bodies = pre.avatars()
bodies.addAvatar(body)
pre.visuAvatars(bodies)

In [ ]:
body=pre.granuloRoughWall3D(center=[0.,0.,0.], lx=2., ly=2., rmin=0.05, rmax=0.1, 
                            model=mod, material=mat, color='WALLx')
bodies = pre.avatars()
bodies.addAvatar(body)
pre.visuAvatars(bodies)